In [ ]:
import json
import nltk.data
import itertools
import random
import numpy as np
import os

In [ ]:
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
SHUFFLE_COUNT = 5
max_seq_len = 10

In [ ]:
os.makedirs("./arxiv_data/original/", exist_ok=True)
os.makedirs("./arxiv_data/permutations/", exist_ok=True)
os.makedirs("./arxiv_data/processed/", exist_ok=True)

In [ ]:
with open("./data/arxivData.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
count = 0
for index,paper in enumerate(data):
    file_name = "./arxiv_data/original/original_"+str(index)+".csv"
    f = open('%s' % file_name, 'w+')
    abstract = " ".join(paper['summary'].strip())
    for line_number,line in enumerate(tokenizer.tokenize(abstract)):
        f.write(str(line_number) + "\t" + line + "\n")
    
    count += line_number
        

    f.close()
    

In [ ]:
for index in range(len(data)):
    file_name = "./arxiv_data/original/original_"+str(index)+".csv"
    f = open('%s' % file_name, 'r')
    
    content = f.readlines()
    for i in range(SHUFFLE_COUNT):
        shuffled_file_name = "./arxiv_data/permutations/original_"+ str(index) +"_permutation_"+str(i)+".csv"
        shuffled_f = open('%s' % shuffled_file_name, 'w+')
        if(i == 0):
            shuffled_content =''.join(content)
        else:
            random.shuffle(content)
            shuffled_content = ''.join(content)
            
        shuffled_f.write(shuffled_content)
        shuffled_f.close()
    
    f.close()

In [ ]:
permutationFiles =  os.listdir("./arvix_data/permutations/")
len(permutationFiles)

In [ ]:
permutationData = []
for file in permutationFiles:
    with open("./arvix_data/permutations/"+file) as f:
        permutationData.append(f.readlines())

In [ ]:
num_files = len(permutationData)

processedData = np.array([[" " for i in range(max_seq_len)] for i in range(num_files)], dtype = object)
processedDataPerm = np.array([[-1 for i in range(max_seq_len)] for i in range(num_files)])
for i in range(num_files):
    fileData = permutationData[i]
    j = 0
    for line in fileData:
        processedData[i][j] = ''.join(line.split('\t')[1:]).strip()
        
        if(int(line.split('\t')[0]) <= max_seq_len):
            processedDataPerm[i][j] = int(line.split('\t')[0])
            j += 1
        if( j >= max_seq_len ):
            break

In [ ]:
with open('./arxiv_data/processed/sentences.txt','w+') as f:
    for line in processedData:
        f.write('\t'.join(line).strip() + '\n')

In [ ]:
with open("./arxiv_data/processed/permutations.txt",'w+') as f:
    for line in processedDataPerm:
        f.write('\t'.join(line.astype(str)).strip() + '\n')